# ALS

## Create Spark Session

In [87]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .appName('Homework9Part1')\
        .config('spark.driver.maxResultSize', '10g')\
        .config('spark.executor.memory' ,'10g')\
        .config('spark.driver.memory', '10g')\
        .getOrCreate()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## Import the train and test dataset
* Train: `trainIdx2_matrix.txt`
* Test: `testTrack_hierarchy.txt`

In [88]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

train_schema = StructType([
    StructField('userId', IntegerType()),
    StructField('itemId', IntegerType()),
    StructField('rating', DoubleType())
])

train = spark.read.csv('RawData/trainIdx2_matrix.txt',
                       sep='|',
                       header=False,
                       schema=train_schema
                      )

train.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [89]:
train.show(5)

+------+------+------+
|userId|itemId|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
|199808| 42563|  90.0|
|199808| 59092|  90.0|
+------+------+------+
only showing top 5 rows



In [90]:
test_schema = StructType([
    StructField('userId', IntegerType()),
    StructField('trackId', IntegerType()),
    StructField('albumId', IntegerType()),
    StructField('artistId', IntegerType()),
    StructField('genreId_1', IntegerType()),
    StructField('genreId_2', IntegerType()),
    StructField('genreId_3', IntegerType()),
    StructField('genreId_4', IntegerType()),
    StructField('genreId_5', IntegerType()),
    StructField('genreId_6', IntegerType()),
    StructField('genreId_7', IntegerType()),
    StructField('genreId_8', IntegerType()),
    StructField('genreId_9', IntegerType()),
    StructField('genreId_10', IntegerType()),
    StructField('genreId_11', IntegerType()),
    StructField('genreId_12', IntegerType()),
    StructField('genreId_13', IntegerType()),
    StructField('genreId_14', IntegerType()),
    StructField('genreId_15', IntegerType()),
    StructField('genreId_16', IntegerType()),
    StructField('genreId_17', IntegerType()),
    StructField('genreId_18', IntegerType()),
    StructField('genreId_19', IntegerType()),
    StructField('genreId_20', IntegerType()),
    StructField('genreId_21', IntegerType()),
])

test = spark.read.csv('RawData/testTrack_hierarchy.txt',
                     sep='|',
                     nullValue='None',
                     header=False,
                     schema=test_schema)

test.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- trackId: integer (nullable = true)
 |-- albumId: integer (nullable = true)
 |-- artistId: integer (nullable = true)
 |-- genreId_1: integer (nullable = true)
 |-- genreId_2: integer (nullable = true)
 |-- genreId_3: integer (nullable = true)
 |-- genreId_4: integer (nullable = true)
 |-- genreId_5: integer (nullable = true)
 |-- genreId_6: integer (nullable = true)
 |-- genreId_7: integer (nullable = true)
 |-- genreId_8: integer (nullable = true)
 |-- genreId_9: integer (nullable = true)
 |-- genreId_10: integer (nullable = true)
 |-- genreId_11: integer (nullable = true)
 |-- genreId_12: integer (nullable = true)
 |-- genreId_13: integer (nullable = true)
 |-- genreId_14: integer (nullable = true)
 |-- genreId_15: integer (nullable = true)
 |-- genreId_16: integer (nullable = true)
 |-- genreId_17: integer (nullable = true)
 |-- genreId_18: integer (nullable = true)
 |-- genreId_19: integer (nullable = true)
 |-- genreId_20: integer (n

In [91]:
test.select('userId', 'trackId', 'artistId', 'genreId_1', 'genreId_2', 'genreId_3', 'genreId_4', 'genreId_5').show(5)

+------+-------+--------+---------+---------+---------+---------+---------+
|userId|trackId|artistId|genreId_1|genreId_2|genreId_3|genreId_4|genreId_5|
+------+-------+--------+---------+---------+---------+---------+---------+
|199810| 208019|    null|     null|     null|     null|     null|     null|
|199810|  74139|  271146|   113360|   173467|   173655|   192976|   146792|
|199810|   9903|    null|    33722|   123396|    79926|    73523|     null|
|199810| 242681|  244574|    61215|    17453|   274088|     null|     null|
|199810|  18515|   33168|    19913|    48505|   154024|     null|     null|
+------+-------+--------+---------+---------+---------+---------+---------+
only showing top 5 rows



## Only include the userIds that exist in the test dataset within the train dataset for the ALS model

### Get unique users

In [92]:
test_unique_users = test.select('userId').distinct().coalesce(1)

test_unique_users.show(5)
print('The number of unique users: ', test_unique_users.count())

+------+
|userId|
+------+
|199855|
|199976|
|200166|
|200625|
|200878|
+------+
only showing top 5 rows

The number of unique users:  20000


### Filter the trainset to only include userIds that are included within the test dataset

In [93]:
from pyspark.sql.functions import col

train = train.toPandas()

train = train[train.userId.isin(test_unique_users.toPandas().userId)]

train = spark.createDataFrame(train).repartition('userId')

In [94]:
train.printSchema()

root
 |-- userId: long (nullable = true)
 |-- itemId: long (nullable = true)
 |-- rating: double (nullable = true)



In [95]:
train.show(5)

+------+------+------+
|userId|itemId|rating|
+------+------+------+
|200309| 88934| 100.0|
|200309|293425| 100.0|
|200309| 69014|  10.0|
|200309| 28342|  80.0|
|200309| 28964|  80.0|
+------+------+------+
only showing top 5 rows



## Train data summary statistics

In [96]:
train.describe().show()

+-------+------------------+-----------------+------------------+
|summary|            userId|           itemId|            rating|
+-------+------------------+-----------------+------------------+
|  count|          10643437|         10643437|          10643437|
|   mean|224380.43626321084|149126.9231043506|47.600189769526516|
| stddev|14393.139199046373| 85467.7984951211|37.996779529988316|
|    min|            199810|                0|               0.0|
|    max|            249010|           296110|             100.0|
+-------+------------------+-----------------+------------------+



## Build ALS Model

In [97]:
from pyspark.ml.recommendation import ALS

als = ALS(userCol='userId', 
          itemCol='itemId',
          ratingCol='rating', 
          rank=5,
          maxIter= 5,
          regParam=0.01,
          nonnegative = True, 
          implicitPrefs = False)

### Train the best model

In [98]:
model = als.fit(train)

In [99]:
train_results = model.transform(train)

In [100]:
train_results.show(5)

+------+------+------+----------+
|userId|itemId|rating|prediction|
+------+------+------+----------+
|205890|   148|  90.0|  71.39319|
|216277|   148|  90.0|  67.19637|
|241707|   148|  60.0| 61.770355|
|226963|   148|  90.0|  83.08943|
|206707|   148|  70.0| 65.531624|
+------+------+------+----------+
only showing top 5 rows



### Training summary

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

print('RMSE: ', evaluator.evaluate(train_results))

RMSE:  28.15705746813551


In [16]:
train_results.select('rating').describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          10643437|
|   mean|47.600189769526516|
| stddev|  37.9967795299883|
|    min|               0.0|
|    max|             100.0|
+-------+------------------+



## Make predictions for track, album, and artist on the test data

### Check for null values within `trackId`, `albumId`, and `artistId` columns

In [17]:
print('Number of null trackId: ', test.filter('trackId IS NULL').count())
print('Number of null albumId: ', test.filter('albumId IS NULL').count())
print('Number of null artistId: ', test.filter('artistId IS NULL').count())

Number of null trackId:  0
Number of null albumId:  8572
Number of null artistId:  10891


### Make predictions on track ratings

In [18]:
prediction_track = model.setItemCol('trackId').transform(test)

### Make predictions on album ratings

In [19]:
prediction_album = model.setItemCol('albumId').transform(test.filter('albumId IS NOT NULL'))

### Make predictions on artist ratings 

In [20]:
prediction_artist = model.setItemCol('artistId').transform(test.filter('artistId IS NOT NULL'))

## Add all ratings to the same dataframe, `predictions`

In [21]:
from pyspark.sql.types import IntegerType

predictions = test.select('userId', 'trackId')

predictions = predictions.join(prediction_track.select('userId', 'trackId', 'prediction'), ['userId', 'trackId'], 'left')
predictions = predictions.withColumn('prediction', predictions['prediction'].cast(IntegerType()))\
                         .withColumnRenamed('prediction', 'track_rating')

In [22]:
predictions = predictions.join(prediction_album.select('userId', 'trackId', 'prediction'), ['userId', 'trackId'], 'left')

predictions = predictions.withColumn('prediction', predictions['prediction'].cast(IntegerType()))\
                         .withColumnRenamed('prediction', 'album_rating')

In [23]:
predictions = predictions.join(prediction_artist.select('userId', 'trackId', 'prediction'), ['userId', 'trackId'], 'left')

predictions = predictions.withColumn('prediction', predictions['prediction'].cast(IntegerType()))\
                         .withColumnRenamed('prediction', 'artist_rating')

In [24]:
predictions.show(5)

+------+-------+------------+------------+-------------+
|userId|trackId|track_rating|album_rating|artist_rating|
+------+-------+------------+------------+-------------+
|200072|  29894|          75|          73|           76|
|200124| 162126|           9|          10|           11|
|200174| 137908|          25|          40|           35|
|200400| 263168|          71|         100|           67|
|200427|  82634|          10|          85|          116|
+------+-------+------------+------------+-------------+
only showing top 5 rows



### Check for null values within the `track_rating`, `album_rating`, and `artist_rating`

In [25]:
print('Number of null track_rating: ', predictions.filter('track_rating IS NULL').count())
print('Number of null album_rating: ', predictions.filter('album_rating IS NULL').count())
print('Number of null artist_rating: ', predictions.filter('artist_rating IS NULL').count())

Number of null track_rating:  0
Number of null album_rating:  8572
Number of null artist_rating:  10891


### Replace null values within `album_rating` and `artist_rating` with `0`

In [26]:
predictions = predictions.na.fill(0)

## Count the number of genres per userId-trackId pair

In [27]:
test_genres = test.select('userId', 
                          'trackId', 
                          'genreId_1', 
                          'genreId_2', 
                          'genreId_3', 
                          'genreId_4', 
                          'genreId_5', 
                          'genreId_6', 
                          'genreId_7', 
                          'genreId_8', 
                          'genreId_9', 
                          'genreId_10', 
                          'genreId_11', 
                          'genreId_12', 
                          'genreId_13', 
                          'genreId_14',
                          'genreId_15',
                          'genreId_16',
                          'genreId_17',
                          'genreId_18',
                          'genreId_19',
                          'genreId_20',
                          'genreId_21')

In [28]:
from pyspark.sql.functions import isnull

num_genres = test_genres.select('userId', 'trackId', sum([isnull(test_genres[col]).cast(IntegerType()) for col in test_genres.columns]).alias('num_genres'))

In [29]:
num_genres.show(5)

+------+-------+----------+
|userId|trackId|num_genres|
+------+-------+----------+
|199810| 208019|        21|
|199810|  74139|        14|
|199810|   9903|        17|
|199810| 242681|        18|
|199810|  18515|        18|
+------+-------+----------+
only showing top 5 rows



### Add the number of genres into `predictions` dataframe, 

In [30]:
predictions = predictions.join(num_genres, ['userId', 'trackId'], 'left')

In [31]:
predictions.show(5)

+------+-------+------------+------------+-------------+----------+
|userId|trackId|track_rating|album_rating|artist_rating|num_genres|
+------+-------+------------+------------+-------------+----------+
|200072|  29894|          75|          73|           76|        16|
|200124| 162126|           9|          10|           11|        18|
|200174| 137908|          25|          40|           35|        16|
|200400| 263168|          71|         100|           67|        20|
|200427|  82634|          10|          85|          116|        18|
+------+-------+------------+------------+-------------+----------+
only showing top 5 rows



## Write `predictions` to csv

In [32]:
predictions.coalesce(1).write.csv('Data/ratings.csv', header=True)

# Create the train matrix that has the same structure of the test data

In [9]:
import pandas as pd 

train2 = pd.read_csv('RawData/trainIdx2_matrix.txt', header=None, sep='|', names=['userId', 'itemId', 'rating'] )

In [10]:
train2.head()

,userId,itemId,rating
0,199808,248969,90
1,199808,2663,90
2,199808,28341,90
3,199808,42563,90
4,199808,59092,90


In [11]:
trackIds = pd.read_csv('RawData/trackData2.txt', sep='|', usecols=[0], header=None, names=['trackId'])
albumIds = pd.read_csv('RawData/albumData2.txt', sep='|', usecols=[0], header=None, names=['albumId'])
artistIds = pd.read_csv('RawData/artistData2.txt', sep='|', usecols=[0], header=None, names=['artistId'])
genreIds = pd.read_csv('RawData/genreData2.txt', sep='|', usecols=[0], header=None, names=['genreId'])

## Classify whether the itemId within the trainset is a track, album, artist, or genre

In [12]:
train_tracks = train2[train2.itemId.isin(trackIds.trackId)]
train_albums = train2[train2.itemId.isin(albumIds.albumId)]
train_artists = train2[train2.itemId.isin(artistIds.artistId)]
train_genres = train2[train2.itemId.isin(genreIds.genreId)]

In [13]:
train_tracks = train_tracks.rename(columns = {'rating': 'track_rating', 'itemId': 'trackId'})
train_albums = train_albums.rename(columns = {'rating': 'album_rating', 'itemId': 'albumId'})
train_artists = train_artists.rename(columns = {'rating': 'artist_rating', 'itemId': 'artistId'})

In [14]:
train_tracks.head()

,userId,trackId,track_rating
87,199810,47420,90
88,199810,158436,50
89,199810,256008,70
90,199810,234891,70
130,199810,150186,70


In [15]:
train_albums.head()

,userId,albumId,album_rating
134,199810,26374,50
138,199810,204650,50
141,199810,9774,50
280,199811,271229,70
291,199812,112725,100


In [16]:
train_artists.head()

,userId,artistId,artist_rating
0,199808,248969,90
1,199808,2663,90
2,199808,28341,90
3,199808,42563,90
4,199808,59092,90


## Get the hierarchy of the tracks and albums 

In [17]:
track_h = pd.read_csv('RawData/trackData2.txt', sep='|', header=None, na_values=['None'], names=['trackId', 'albumId', 'artistId', 'genreId_1', 'genreId_2', 'genreId_3', 'genreId_4', 'genreId_5', 'genreId_6', 'genreId_7', 'genreId_8', 'genreId_9', 'genreId_10', 'genreId_11', 'genreId_12', 'genreId_13', 'genreId_14', 'genreId_15', 'genreId_16', 'genreId_17', 'genreId_18', 'genreId_19', 'genreId_20', 'genreId_21'])
 
album_h = pd.read_csv('RawData/albumData2.txt', sep='|', header=None, na_values=['None'], names=['albumId', 'artistId', 'genreId_1', 'genreId_2', 'genreId_3', 'genreId_4', 'genreId_5', 'genreId_6', 'genreId_7', 'genreId_8', 'genreId_9', 'genreId_10', 'genreId_11', 'genreId_12', 'genreId_13', 'genreId_14', 'genreId_15', 'genreId_16', 'genreId_17', 'genreId_18', 'genreId_19', 'genreId_20', 'genreId_21'])

In [18]:
train_tracks_h = pd.merge(train_tracks, track_h, how='left', on='trackId')

In [19]:
train_albums_h = pd.merge(train_albums, album_h, how='left', on='albumId')

### Only keep the train albums seperate that are not already inside `train_tracks_h`

In [20]:
train_albums_h = train_tracks_h[~train_tracks_h.albumId.isin(train_albums_h.albumId)]

## For the `train_tracks_h` check to see if ratings exist in the other matrices

In [21]:
track_rating_in_albums = train_albums[train_albums.albumId.isin(train_tracks_h.albumId)]

In [22]:
track_rating_in_albums.head()

,userId,albumId,album_rating
134,199810,26374,50
138,199810,204650,50
141,199810,9774,50
280,199811,271229,70
291,199812,112725,100


In [23]:
track_rating_in_artists = train_artists[train_artists.artistId.isin(train_tracks_h.artistId)]

In [24]:
track_rating_in_artists.head()

,userId,artistId,artist_rating
0,199808,248969,90
1,199808,2663,90
2,199808,28341,90
3,199808,42563,90
4,199808,59092,90


### Combine the matrices together

In [25]:
final = pd.merge(train_tracks_h, track_rating_in_albums, how='outer', on=['userId', 'albumId'])

In [26]:
final = pd.merge(final, track_rating_in_artists, how='outer', on=['userId', 'artistId'])

In [27]:
final.head()

,userId,trackId,track_rating,albumId,artistId,genreId_1,genreId_2,genreId_3,genreId_4,genreId_5,...,genreId_14,genreId_15,genreId_16,genreId_17,genreId_18,genreId_19,genreId_20,genreId_21,album_rating,artist_rating
0,199810,47420.0,90.0,190891.0,22907.0,243099.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,199810,158436.0,50.0,121272.0,48050.0,214110.0,181006.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0
2,199810,256008.0,70.0,56953.0,275191.0,158282.0,242383.0,207648.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0
3,199810,234891.0,70.0,18215.0,257019.0,131552.0,47898.0,201738.0,88853.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0
4,199810,265026.0,70.0,252707.0,257019.0,131552.0,201738.0,88853.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0


## Remove columns that have an `Nan` value for trackId since the test set only consists of trackIds

In [28]:
final = final.dropna(subset=['trackId'])

In [163]:
## Save final as csv
final.to_csv('train_with_empty.csv', index=False, na_rep='None', columns=['userId', 'trackId', 'albumId', 'artistId', 'genreId_1', 'genreId_2', 'genreId_3', 'genreId_4', 'genreId_5', 'genreId_6', 'genreId_7', 'genreId_8', 'genreId_9', 'genreId_10', 'genreId_11', 'genreId_12', 'genreId_13', 'genreId_14', 'genreId_15', 'genreId_16', 'genreId_17', 'genreId_18', 'genreId_19', 'genreId_20', 'genreId_21'])

In [190]:
empty_album_ratings = final[final.album_rating.isna()]
empty_album_ratings = empty_album_ratings.dropna(subset=['albumId'])
empty_album_ratings.head()

empty_album_ratings[['userId', 'trackId', 'albumId']].to_csv('empty_album_ratings.csv', index=False)

In [210]:
empty_artist_ratings = final[final.artist_rating.isna()]
empty_artist_ratings = empty_artist_ratings.dropna(subset=['albumId'])
empty_artist_ratings.head()

empty_artist_ratings[['userId', 'trackId', 'artistId']].to_csv('empty_artist_ratings.csv', index=False)

## Make predictions on the empty values

In [222]:
albums = spark.read.csv('empty_album_ratings.csv', sep=',', header=True)
artists = spark.read.csv('empty_artist_ratings.csv', sep=',', header=True)

In [223]:
albums.printSchema()

root
 |-- userId: string (nullable = true)
 |-- trackId: string (nullable = true)
 |-- albumId: string (nullable = true)



In [224]:
artists.printSchema()

root
 |-- userId: string (nullable = true)
 |-- trackId: string (nullable = true)
 |-- artistId: string (nullable = true)



In [225]:
albums = albums.withColumn('userId', albums['userId'].cast(IntegerType()))
albums = albums.withColumn('trackId', albums['trackId'].cast(IntegerType()))
albums = albums.withColumn('albumId', albums['albumId'].cast(IntegerType()))

artists = artists.withColumn('userId', artists['userId'].cast(IntegerType()))
artists = artists.withColumn('trackId', artists['trackId'].cast(IntegerType()))
artists = artists.withColumn('artistId', artists['artistId'].cast(IntegerType()))

In [226]:
albums.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- trackId: integer (nullable = true)
 |-- albumId: integer (nullable = true)



In [227]:
artists.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- trackId: integer (nullable = true)
 |-- artistId: integer (nullable = true)



In [230]:
prediction_album_train = model.transform(albums.filter('albumId IS NOT NULL').withColumnRenamed('albumId', 'itemId')).withColumnRenamed('itemId', 'albumId').withColumnRenamed('prediction', 'album_rating')
prediction_artists_train = model.transform(artists.filter('artistId IS NOT NULL').withColumnRenamed('artistId', 'itemId')).withColumnRenamed('itemId', 'artistId').withColumnRenamed('prediction', 'artist_rating')

prediction_album_train.coalesce(1).write.csv('predicted_albums.csv', header=True)
prediction_artists_train.coalesce(1).write.csv('predicted_artists.csv', header=True)

## Find num genres

In [171]:
final2 = spark.read.csv('train_with_empty.csv',
                     sep=',',
                     nullValue='None',
                     header=True)

In [172]:
final2.show(5)

+------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|userId| trackId| albumId|artistId|genreId_1|genreId_2|genreId_3|genreId_4|genreId_5|genreId_6|genreId_7|genreId_8|genreId_9|genreId_10|genreId_11|genreId_12|genreId_13|genreId_14|genreId_15|genreId_16|genreId_17|genreId_18|genreId_19|genreId_20|genreId_21|
+------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|199810| 47420.0|190891.0| 22907.0| 243099.0|     null|     null|     null|     null|     null|     null|     null|     null|      null|      null|      null|      null|      null|      null|      null|      null|      null|  

In [173]:
final_genres = final2.select('userId', 
                          'trackId', 
                          'genreId_1', 
                          'genreId_2', 
                          'genreId_3', 
                          'genreId_4', 
                          'genreId_5', 
                          'genreId_6', 
                          'genreId_7', 
                          'genreId_8', 
                          'genreId_9', 
                          'genreId_10', 
                          'genreId_11', 
                          'genreId_12', 
                          'genreId_13', 
                          'genreId_14',
                          'genreId_15',
                          'genreId_16',
                          'genreId_17',
                          'genreId_18',
                          'genreId_19',
                          'genreId_20',
                          'genreId_21')

In [174]:
final_num_genres = final_genres.select('userId', 'trackId', sum([isnull(final_genres[col]).cast(IntegerType()) for col in final_genres.columns]).alias('num_genres'))

In [182]:
final_num_genres.show(5)

final_num_genres.coalesce(1).write.csv('train_num_genres.csv', header=True)

+------+--------+----------+
|userId| trackId|num_genres|
+------+--------+----------+
|199810| 47420.0|        20|
|199810|158436.0|        19|
|199810|256008.0|        18|
|199810|234891.0|        17|
|199810|265026.0|        18|
+------+--------+----------+
only showing top 5 rows



In [29]:
predicted_artists = pd.read_csv('TrainPredictions/predicted_artists.csv')
predicted_albums = pd.read_csv('TrainPredictions/predicted_albums.csv')
num_genres_df = pd.read_csv('TrainPredictions/train_num_genres.csv')

In [94]:
predicted_artists = predicted_artists.set_index(['userId', 'trackId'])

In [95]:
predicted_albums = predicted_albums.set_index(['userId', 'trackId'])

In [96]:
num_genred_df = num_genres_df.set_index(['userId', 'trackId'])

In [97]:
final = final[['userId', 'trackId', 'track_rating', 'album_rating', 'artist_rating']].set_index(['userId', 'trackId'])
final.head()

track_rating  album_rating  artist_rating
userId trackId                                            
199810 47420.0           90.0           NaN            NaN
       158436.0          50.0           NaN           70.0
       256008.0          70.0           NaN           50.0
       234891.0          70.0           NaN           90.0
       265026.0          70.0           NaN           90.0

In [100]:
final['artist_rating'] = final['artist_rating'].fillna(predicted_artists['artist_rating'])

In [101]:
final['album_rating'] = final['album_rating'].fillna(predicted_albums['album_rating'])

In [102]:
final['num_genres'] = num_genred_df['num_genres']

In [103]:
final.head()

track_rating  album_rating  artist_rating  num_genres
userId trackId                                                        
199810 47420.0           90.0     53.398773       50.37487          20
       158436.0          50.0     57.158050       70.00000          19
       256008.0          70.0     69.713486       50.00000          18
       234891.0          70.0     53.166330       90.00000          17
       265026.0          70.0     54.305294       90.00000          18

In [104]:
final.isna().sum()

track_rating          0
album_rating     506376
artist_rating    585125
num_genres            0
dtype: int64

In [108]:
final = final.fillna(0)

In [109]:
final.isna().sum()

track_rating     0
album_rating     0
artist_rating    0
num_genres       0
dtype: int64

In [110]:
final.to_csv('finalTrainset.csv')